In [1]:
import sys
import logging

import numpy as np
import scipy as sp

%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from PIL import Image

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, io, transforms

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


In [3]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained("efficientnet-b3")

# ... image preprocessing as in the classification example ...
tfms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)
img = tfms(Image.open("../food/00002.jpg")).unsqueeze(0)
print(img.shape)  # torch.Size([1, 3, 224, 224])

phi1 = model.extract_features(img)
print(phi1.squeeze().shape)  # torch.Size([1, 1280, 7, 7])


Loaded pretrained weights for efficientnet-b3
torch.Size([1, 3, 224, 224])
torch.Size([1536, 7, 7])


In [25]:
embedding_dim = 1536
n_samples = 100_000

tfms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

pretrained = True
efficientnet = models.efficientnet_b3(pretrained=pretrained)
embedding = nn.Sequential(*list(efficientnet.children())[:-1])

# embeddings
embeddings = np.empty((n_samples, embedding_dim))


# with torch.no_grad():
#     for i in range(n_samples + 1):
#         path = '../food/' + str(i).rjust(5, '0') + '.jpg'
#         embeddings[i] = embedding(tfms(Image.open(path)).unsqueeze(0)).squeeze().detach().numpy()

import pickle

filename = "../embeddings.pkl"
# outfile = open(filename, "wb")
# pickle.dump(embeddings, outfile)
# outfile.close()


In [3]:
embedding_dim = 1536
n_samples = 2*59515

embeddings = np.random.random((100_000, 1536))
X = np.random.random((n_samples, 3*embedding_dim))
y = np.random.choice([0,1],n_samples)

# triplets = []
# for line in open("../train_triplets.txt"):
#     triplet = line.strip().split(" ")
#     triplets.append(triplet)

# for i, triplet in enumerate(triplets):
#     a, b, c = [int(i) for i in triplet]
#     X[2*i], y[2*i] = np.concatenate((embeddings[a], embeddings[b], embeddings[c])), 1
#     X[2*i + 1], y[2*i + 1] = np.concatenate((embeddings[a], embeddings[c], embeddings[b])), 0

In [4]:
from sklearn.ensemble import HistGradientBoostingClassifier

params = {"loss": "binary_crossentropy",
    "early_stopping": True,
    "scoring": "roc_auc",
    "l2_regularization": 0.1,
        "learning_rate": 0.2,
        "max_depth": 3,

}

hgbc = HistGradientBoostingClassifier(
    **params, verbose=2
)
hgbc.fit(X, y)  # fit

Binning 3.949 GB of training data: 118.387 s
Binning 0.439 GB of validation data: 3.518 s
Fitting gradient boosted rounds:
[1/100] 1 tree, 8 leaves, max depth = 3, train score: 0.51164, val score: 0.49888, in 0.559s
[2/100] 1 tree, 8 leaves, max depth = 3, train score: 0.51457, val score: 0.50140, in 0.359s
[3/100] 1 tree, 8 leaves, max depth = 3, train score: 0.51969, val score: 0.49889, in 0.353s
[4/100] 1 tree, 8 leaves, max depth = 3, train score: 0.52839, val score: 0.49641, in 0.369s
[5/100] 1 tree, 8 leaves, max depth = 3, train score: 0.53658, val score: 0.49378, in 0.338s
[6/100] 1 tree, 8 leaves, max depth = 3, train score: 0.54248, val score: 0.49527, in 0.348s
[7/100] 1 tree, 8 leaves, max depth = 3, train score: 0.55057, val score: 0.49727, in 0.334s
[8/100] 1 tree, 8 leaves, max depth = 3, train score: 0.55545, val score: 0.50301, in 0.368s
[9/100] 1 tree, 8 leaves, max depth = 3, train score: 0.55865, val score: 0.50664, in 0.371s
[10/100] 1 tree, 8 leaves, max depth = 3

HistGradientBoostingClassifier(early_stopping=True, l2_regularization=0.1,
                               learning_rate=0.2, loss='binary_crossentropy',
                               max_depth=3, scoring='roc_auc', verbose=2)